In [23]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import shutil
import pandas as pd

In [3]:
%cd /Users/ramir/OneDrive/Desktop/MDS_UBC_2021/labs_block6/525

C:\Users\ramir\OneDrive\Desktop\MDS_UBC_2021\labs_block6\525


In [14]:
# Necessary metadata
article_id = 14096681 
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_dory = "figrainfall/"

In [18]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]             
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [19]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 12.3 s
Wall time: 1min 34s


In [20]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 24.6 s
Wall time: 2min 11s


In [22]:
%ls -ltr figrainfall

 Volume in drive C is OS
 Volume Serial Number is 4AFF-EDFD

 Directory of C:\Users\ramir\OneDrive\Desktop\MDS_UBC_2021\labs_block6\525


 Directory of C:\Users\ramir\OneDrive\Desktop\MDS_UBC_2021\labs_block6\525\figrainfall

03/28/2022  02:38 PM    <DIR>          .
03/28/2022  02:38 PM    <DIR>          ..
03/28/2022  02:38 PM    <DIR>          __MACOSX
03/28/2022  02:36 PM       127,613,760 ACCESS-CM2_daily_rainfall_NSW.csv
03/28/2022  02:37 PM       114,707,410 ACCESS-ESM1-5_daily_rainfall_NSW.csv
03/28/2022  02:36 PM        94,960,113 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
03/28/2022  02:37 PM       294,260,911 BCC-CSM2-MR_daily_rainfall_NSW.csv
03/28/2022  02:38 PM        55,224,437 BCC-ESM1_daily_rainfall_NSW.csv
03/28/2022  02:38 PM        46,286,371 CanESM5_daily_rainfall_NSW.csv
03/28/2022  02:37 PM       330,360,682 CMCC-CM2-HR4_daily_rainfall_NSW.csv
03/28/2022  02:38 PM       328,787,320 CMCC-CM2-SR5_daily_rainfall_NSW.csv
03/28/2022  02:37 PM       328,852,379 CMCC-ESM2_dai

File Not Found


In [26]:
# moving/excluding observed rain_fall to another folder 

file = 'observed_daily_rainfall_SYD.csv'
os.makedirs(os.path.join(output_directory, 'observed_daily_rainfall_SYD'), exist_ok=True)
shutil.move(os.path.join(output_directory, file), os.path.join(output_directory, 'observed_daily_rainfall_SYD', file))

'figrainfall/observed_daily_rainfall_SYD\\observed_daily_rainfall_SYD.csv'

#### 1. Combine data CSVs into a single CSV using pandas.

In [30]:
files = glob.glob(f"{output_directory}/*.csv")
                  
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(year=re.findall("^[^_]*", file)[0])
                for file in files)
              )
df.to_csv(f"{output_directory}combined_data.csv")

In [ ]:
data = pd.read_csv(f"{output_directory}combined_data.csv")

In [ ]:
data.